In [41]:
# Libraries and dependencies
import os
import alpaca_trade_api as tradeapi
import pandas as pd 
from pathlib import Path
from dotenv import load_dotenv

%matplotlib inline

In [42]:
# Loading .env 
load_dotenv()

True

In [43]:
# Set Alpaca API key/ secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Conection
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2")

In [44]:
# Gathering data with the api conection and creating a dataframe
# Set ticekers
tickers_crypto = ["BTC", "ETH", "BNB", "ADA", "XRP", "LTC", "LINK", "BCH", "VET", "XLM"]
tickers_stocks = ["MSFT", "AAPL", "TSLA", "AMZN", "NVDA", "GOOG", "FB", "ADBE", "NFLX", "PYPL"]

# Setting time frame to 1D
timeframe = "1D"

# Set the start/ end datetime of 5 years from today 
start_date = pd.Timestamp("2019-11-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp.now(tz="America/New_York").isoformat()

# Set limit of rows to the max to maximaize
limit_rows = 1000

# Get the 2 years data frame for stocks
stocks_df = api.get_barset(
    tickers_stocks,
    timeframe,
    start = start_date,
    end = end_date,
    limit = limit_rows).df

# Visualazing the data 
stocks_df.head()

AAPL                                        ADBE  \
                             open     high      low   close    volume    open   
time                                                                            
2019-11-01 00:00:00-04:00  249.54  255.930  249.160  255.84  35281756  279.14   
2019-11-04 00:00:00-05:00  257.33  257.845  255.380  257.48  23618001  280.56   
2019-11-05 00:00:00-05:00  257.05  258.190  256.320  257.15  17738372  292.20   
2019-11-06 00:00:00-05:00  256.84  257.490  255.365  257.24  15328064  289.11   
2019-11-07 00:00:00-05:00  258.74  260.350  258.110  259.50  22267200  288.30   

                                                              ...    PYPL  \
                             high       low   close   volume  ...    open   
time                                                          ...           
2019-11-01 00:00:00-04:00  281.83  277.2675  277.76  2413757  ...  104.70   
2019-11-04 00:00:00-05:00  281.21  276.0500  277.62  2173646  ...  105.72   
2019-11-05 00:00:00-05:00  292.88  283.6000  289.32  5717319  ...  103.05   
2019-11-06 00:00:00-05:00  289.48  285.6100  287.36  2370563  ...  101.20   
2019-11-07 00:00:00-05:00  293.48  286.9950  290.81  2664860  ...  100.94   

                                                               TSLA          \
                             high      low   close   volume    open    high   
time                                                                          
2019-11-01 00:00:00-04:00  105.30  103.930  105.00  4067334  316.32  316.48   
2019-11-04 00:00:00-05:00  105.76  102.605  102.79  4885673  314.80  321.94   
2019-11-05 00:00:00-05:00  103.28  100.285  101.00  8202861  319.62  323.51   
2019-11-06 00:00:00-05:00  101.34  100.170  100.62  5612352  318.00  326.72   
2019-11-07 00:00:00-05:00  101.72  100.290  100.47  7545836  329.14  341.50   

                                                       
                               low    close    volume  
time                                                   
2019-11-01 00:00:00-04:00  309.800  313.210   6178310  
2019-11-04 00:00:00-05:00  309.260  317.425   8579754  
2019-11-05 00:00:00-05:00  316.118  317.160   6308975  
2019-11-06 00:00:00-05:00  314.500  326.700   7748529  
2019-11-07 00:00:00-05:00  328.020  335.390  14148570  

[5 rows x 50 columns]

In [45]:
#Create for loop to read in Change % data and clean column of string values and convert to float /100
crypto_daily_df = pd.DataFrame()
for crypto in crypto_tickers:
    df = pd.read_csv(Path(f"Crypto_Historical_Data/{crypto}.csv"), index_col="Date", parse_dates=True, infer_datetime_format=True)
    #If dtype is O ---object then str.replace
    if df['Change %'].dtypes=='O':
        df['Change %'] = df['Change %'].str.replace('%','')
    else:
        df['Change %'] = df['Change %'].replace('%','')
    crypto_daily_df[f'{crypto}'] = df['Change %'].astype('float').divide(100)

#drop na values or change to 0 depending on need
crypto_daily_df = crypto_daily_df.dropna()

display(crypto_daily_df.head())
display(crypto_daily_df.tail())

,BTC,ETH,BNB,ADA,XRP,DOGE,LTC,LINK,BCH,XLM
Date,,,,,,,,,,
2021-04-25,-0.0225,0.0454,0.0137,-0.0111,-0.0155,-0.0700,0.0047,0.0236,-0.0185,-0.0211
2021-04-24,-0.0206,-0.0639,-0.0552,-0.0473,-0.1026,0.0856,-0.0671,-0.1119,-0.0800,-0.0560
2021-04-23,-0.0113,-0.0141,0.0410,0.0092,0.0098,-0.0463,-0.0458,-0.0029,-0.0177,0.0008
2021-04-22,-0.0388,0.0171,-0.0690,-0.0483,-0.1020,-0.1467,-0.0189,-0.0288,-0.0610,-0.0840
2021-04-21,-0.0471,0.0123,-0.0752,-0.0496,-0.0690,-0.0380,-0.0119,-0.0697,-0.0330,-0.0708


,BTC,ETH,BNB,ADA,XRP,DOGE,LTC,LINK,BCH,XLM
Date,,,,,,,,,,
2018-02-06,0.1099,0.1248,0.2307,0.1325,0.1182,0.2460,0.1351,0.0622,0.0930,0.1224
2018-02-05,-0.1561,-0.1598,-0.2025,-0.1702,-0.1670,-0.1958,-0.1528,-0.0882,-0.2379,-0.1379
2018-02-04,-0.1103,-0.1462,-0.1387,-0.1767,-0.1478,-0.1437,-0.0813,-0.1426,-0.0905,-0.1611
2018-02-03,0.0391,0.0550,0.0000,0.1600,0.0673,0.1279,0.2230,0.0348,0.0677,0.0783
2018-02-02,-0.0314,-0.1090,0.0247,-0.0515,-0.0547,-0.1093,-0.0765,-0.0410,-0.0639,-0.0859


In [46]:
# Cleaning the data from stocks
clean_df_stocks = stocks_df.dropna()#.isnull().sum()
clean_df_stocks.head()

AAPL                                        ADBE  \
                             open     high      low   close    volume    open   
time                                                                            
2019-11-01 00:00:00-04:00  249.54  255.930  249.160  255.84  35281756  279.14   
2019-11-04 00:00:00-05:00  257.33  257.845  255.380  257.48  23618001  280.56   
2019-11-05 00:00:00-05:00  257.05  258.190  256.320  257.15  17738372  292.20   
2019-11-06 00:00:00-05:00  256.84  257.490  255.365  257.24  15328064  289.11   
2019-11-07 00:00:00-05:00  258.74  260.350  258.110  259.50  22267200  288.30   

                                                              ...    PYPL  \
                             high       low   close   volume  ...    open   
time                                                          ...           
2019-11-01 00:00:00-04:00  281.83  277.2675  277.76  2413757  ...  104.70   
2019-11-04 00:00:00-05:00  281.21  276.0500  277.62  2173646  ...  105.72   
2019-11-05 00:00:00-05:00  292.88  283.6000  289.32  5717319  ...  103.05   
2019-11-06 00:00:00-05:00  289.48  285.6100  287.36  2370563  ...  101.20   
2019-11-07 00:00:00-05:00  293.48  286.9950  290.81  2664860  ...  100.94   

                                                               TSLA          \
                             high      low   close   volume    open    high   
time                                                                          
2019-11-01 00:00:00-04:00  105.30  103.930  105.00  4067334  316.32  316.48   
2019-11-04 00:00:00-05:00  105.76  102.605  102.79  4885673  314.80  321.94   
2019-11-05 00:00:00-05:00  103.28  100.285  101.00  8202861  319.62  323.51   
2019-11-06 00:00:00-05:00  101.34  100.170  100.62  5612352  318.00  326.72   
2019-11-07 00:00:00-05:00  101.72  100.290  100.47  7545836  329.14  341.50   

                                                       
                               low    close    volume  
time                                                   
2019-11-01 00:00:00-04:00  309.800  313.210   6178310  
2019-11-04 00:00:00-05:00  309.260  317.425   8579754  
2019-11-05 00:00:00-05:00  316.118  317.160   6308975  
2019-11-06 00:00:00-05:00  314.500  326.700   7748529  
2019-11-07 00:00:00-05:00  328.020  335.390  14148570  

[5 rows x 50 columns]

In [48]:
#Calculate the daily change for the stocks_df using for loop
stocks_daily_df = pd.DataFrame()
for stocks in stock_tickers:
    stocks_daily_df[f'{stocks}'] = clean_df_stocks[stocks]['close'].pct_change().dropna()

display(stocks_daily_df.head())
display(stocks_daily_df.tail())

,MSFT,AAPL,TSLA,AMZN,NVDA,GOOG,FB,ADBE,NFLX,PYPL
time,,,,,,,,,,
2019-11-04 00:00:00-05:00,0.005566,0.006410,0.013457,0.007703,0.038621,0.012923,0.005318,-0.000504,0.020960,-0.021048
2019-11-05 00:00:00-05:00,-0.000415,-0.001282,-0.000835,-0.001978,-0.004464,0.000658,-0.002003,0.042144,-0.016929,-0.017414
2019-11-06 00:00:00-05:00,-0.002769,0.000350,0.030079,-0.003197,-0.009684,-0.000015,-0.014357,-0.006775,0.001944,-0.003762
2019-11-07 00:00:00-05:00,0.001527,0.008786,0.026599,-0.004310,0.003854,0.013206,-0.005847,0.012006,0.003430,-0.001491
2019-11-08 00:00:00-05:00,0.011644,0.002466,0.005158,-0.001297,-0.003023,0.001872,0.002416,0.005674,0.006872,0.009655


,MSFT,AAPL,TSLA,AMZN,NVDA,GOOG,FB,ADBE,NFLX,PYPL
time,,,,,,,,,,
2021-11-05 00:00:00-04:00,-0.001040,0.001855,-0.006741,0.011389,-0.001275,0.003568,0.015328,-0.017705,-0.033932,-0.010560
2021-11-08 00:00:00-05:00,0.002440,-0.005223,-0.048067,-0.008175,0.034907,0.000975,-0.007622,0.005993,0.008858,0.015942
2021-11-09 00:00:00-05:00,-0.002968,0.002526,-0.119735,0.025735,-0.004318,-0.000656,-0.009246,0.002281,0.006908,-0.104780
2021-11-10 00:00:00-05:00,-0.015182,-0.019293,0.043234,-0.026706,-0.040136,-0.017657,-0.023107,-0.030857,-0.013767,-0.003610
2021-11-11 00:00:00-05:00,0.004928,-0.000270,-0.003980,-0.002770,0.032303,0.000907,0.000305,-0.006396,0.016231,-0.012754


In [49]:
#change df to have the same datetime format and tz and create a merged_df 
#stocks_daily_df.index = stocks_daily_df.index.tz_convert(None)
stocks_daily_df.index = pd.to_datetime(stocks_daily_df.index).date
crypto_daily_df.index = pd.to_datetime(crypto_daily_df.index).date
daily_df_merged = stocks_daily_df.merge(crypto_daily_df, how='inner', left_index=True, right_index=True)
daily_df_merged

,MSFT,AAPL,TSLA,AMZN,NVDA,GOOG,FB,ADBE,NFLX,PYPL,BTC,ETH,BNB,ADA,XRP,DOGE,LTC,LINK,BCH,XLM
2019-11-04,0.005566,0.006410,0.013457,0.007703,0.038621,0.012923,0.005318,-0.000504,0.020960,-0.021048,0.0215,0.0229,0.0224,0.0350,0.0293,-0.0713,0.0522,0.0090,-0.0012,0.1383
2019-11-05,-0.000415,-0.001282,-0.000835,-0.001978,-0.004464,0.000658,-0.002003,0.042144,-0.016929,-0.017414,-0.0091,0.0151,0.0000,0.0226,0.0060,-0.0089,0.0292,-0.0031,0.0068,0.0605
2019-11-06,-0.002769,0.000350,0.030079,-0.003197,-0.009684,-0.000015,-0.014357,-0.006775,0.001944,-0.003762,0.0030,0.0138,0.0014,0.0148,0.0319,0.0414,0.0152,0.0263,0.0427,-0.0725
2019-11-07,0.001527,0.008786,0.026599,-0.004310,0.003854,0.013206,-0.005847,0.012006,0.003430,-0.001491,-0.0124,-0.0227,-0.0120,-0.0307,-0.0632,0.0094,-0.0425,-0.0219,-0.0435,-0.0348
2019-11-08,0.011644,0.002466,0.005158,-0.001297,-0.003023,0.001872,0.002416,0.005674,0.006872,0.009655,-0.0483,-0.0156,-0.0337,-0.0287,-0.0518,0.0140,-0.0165,0.0114,-0.0472,-0.0418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-19,-0.006827,0.005747,-0.033496,-0.007434,-0.034812,0.002685,-0.012868,-0.016778,0.014030,-0.006670,-0.0100,-0.0342,0.0468,-0.0635,-0.0702,0.2649,-0.0449,-0.0849,-0.0836,-0.0824
2021-04-20,-0.002626,-0.013349,0.005301,-0.011636,-0.012211,-0.004801,0.001323,-0.003796,-0.008857,-0.013505,0.0150,0.0794,0.1657,0.0598,0.0588,-0.2201,-0.0013,0.0903,0.0563,0.0585
2021-04-21,0.009021,0.002779,0.034948,0.008049,0.012491,0.000366,-0.004097,-0.003889,-0.073784,0.001626,-0.0471,0.0123,-0.0752,-0.0496,-0.0690,-0.0380,-0.0119,-0.0697,-0.0330,-0.0708
2021-04-22,-0.013085,-0.011611,-0.032437,-0.015901,-0.033218,-0.011222,-0.016224,-0.009799,0.001002,-0.009779,-0.0388,0.0171,-0.0690,-0.0483,-0.1020,-0.1467,-0.0189,-0.0288,-0.0610,-0.0840


In [50]:
#Calculate Standard Deviation for stocks and cryptos
stocks_standard_deviation = stocks_daily_df.std()
crypto_standard_deviation = crypto_daily_df.std()
display(stocks_standard_deviation)
display(crypto_standard_deviation)

MSFT    0.021377
AAPL    0.040309
TSLA    0.057632
AMZN    0.020103
NVDA    0.045918
GOOG    0.019854
FB      0.023938
ADBE    0.023746
NFLX    0.025149
PYPL    0.028148
dtype: float64

BTC     0.039263
ETH     0.050653
BNB     0.058845
ADA     0.060871
XRP     0.061960
DOGE    1.808321
LTC     0.054113
LINK    0.073293
BCH     0.064402
XLM     0.062168
dtype: float64

In [51]:
#calculate annual standard deviation for stocks and cryptos
trading_days = 252
stocks_annual_std = stocks_standard_deviation * trading_days ** (1/2)
crypto_annual_std = crypto_standard_deviation * trading_days ** (1/2)
display(stocks_annual_std)
display(crypto_annual_std)

MSFT    0.339346
AAPL    0.639889
TSLA    0.914884
AMZN    0.319132
NVDA    0.728922
GOOG    0.315169
FB      0.380011
ADBE    0.376957
NFLX    0.399226
PYPL    0.446828
dtype: float64

BTC      0.623278
ETH      0.804098
BNB      0.934143
ADA      0.966292
XRP      0.983578
DOGE    28.706212
LTC      0.859022
LINK     1.163498
BCH      1.022346
XLM      0.986881
dtype: float64

In [52]:
# Calculate the annual average return data for stocks and crypto
stocks_annual_avg_return = stocks_daily_df.mean()* trading_days
crypto_annual_avg_return = crypto_daily_df.mean()*trading_days

display(stocks_annual_avg_return)
display(crypto_annual_avg_return)

MSFT    0.473111
AAPL    0.099656
TSLA    1.229288
AMZN    0.378638
NVDA    0.644449
GOOG    0.462990
FB      0.332825
ADBE    0.486802
NFLX    0.489747
PYPL    0.423789
dtype: float64

BTC      0.557221
ETH      0.504919
BNB      1.272034
ADA      0.667469
XRP      0.481963
DOGE    14.967133
LTC      0.466788
LINK     1.556885
BCH      0.407838
XLM      0.440476
dtype: float64

In [53]:
# Calculate the annualized Sharpe Ratios for stocks and crypto
stocks_annual_sharpe_ratio = stocks_annual_avg_return/stocks_annual_std
crypto_annual_sharpe_ratio = crypto_annual_avg_return/crypto_annual_std

display(stocks_annual_sharpe_ratio)
display(crypto_annual_sharpe_ratio)

MSFT    1.394186
AAPL    0.155740
TSLA    1.343655
AMZN    1.186463
NVDA    0.884113
GOOG    1.469021
FB      0.875828
ADBE    1.291400
NFLX    1.226741
PYPL    0.948438
dtype: float64

BTC     0.894017
ETH     0.627932
BNB     1.361712
ADA     0.690752
XRP     0.490010
DOGE    0.521390
LTC     0.543394
LINK    1.338108
BCH     0.398924
XLM     0.446332
dtype: float64

In [54]:
#for loop to calculate covariance for each stock to each crypto
covariance_df = pd.DataFrame()
for stock in stock_tickers:
    for crypto in crypto_tickers:
        covariance_df[f'{crypto} cov {stock}'] = daily_df_merged[crypto].cov(daily_df_merged[stock])
covariance_df.head()

,BTC cov MSFT,ETH cov MSFT,BNB cov MSFT,ADA cov MSFT,XRP cov MSFT,DOGE cov MSFT,LTC cov MSFT,LINK cov MSFT,BCH cov MSFT,XLM cov MSFT,...,BTC cov PYPL,ETH cov PYPL,BNB cov PYPL,ADA cov PYPL,XRP cov PYPL,DOGE cov PYPL,LTC cov PYPL,LINK cov PYPL,BCH cov PYPL,XLM cov PYPL
